# Linear Regression

Imports and Helper Functions
---

In [99]:
%matplotlib inline
from ipywidgets import interactive_output
import ipywidgets as widgets
import numpy as np
from matplotlib import pyplot as plt
from sklearn import linear_model

## Data Set Generation

In [100]:
N = 100
datasets={}

X = np.vstack((2.5*np.random.randn(int(N/2),2)-5, (2.5*np.random.randn(int(N/2),2)+5)))
Y = np.hstack((-1*np.ones((int(N/2))), np.ones((int(N/2))))) 

ind = np.random.permutation(range(N))

datasets["Dataset 1"]=(X[ind,:],Y[ind])

## Define Prediction, Loss, and Learning Function

In [127]:
def discriminant(X,w0,w1,b):
  Yhat = w0*X[:,0] + w1*X[:,1] +b
  return Yhat

def classification_error(Y,Yhat):
  #Compute the empirical risk given Y and Yhat
  R = np.mean(Y*Yhat<0)
  return(R)

def fit(X,Y):
  clf = linear_model.LogisticRegression(C=1e10)
  clf.fit(X,Y)
  w0=clf.coef_[0,0]
  w1=clf.coef_[0,1]
  b=clf.intercept_
  return(w0,w1,b)


## Interaction Functions and UI

In [146]:
def plot_model(w0=0, w1=0, b=0, dataset=None, N=0, learn=True):

    plt.figure(figsize=(10,6))
    plt.rcParams.update({'font.size': 12})

    if(dataset is not None and N>0):
      X,Y=datasets[dataset]
      X=X[:N]
      Y=Y[:N]
    
      ind1=(Y==1)
      indm1=(Y==-1)
      plt.plot(X[ind1,0],X[ind1,1],'bs')
      plt.plot(X[indm1,0],X[indm1,1],'rs')

      if(learn and np.sum(ind1)>0 and np.sum(indm1)>0):
        w0,w1,b=fit(X,Y)
    
      Yhat = predict(X,w0,w1,b)
      R = classification_error(Y,Yhat)
    
    else:
      R=0

    gx1, gx2 = np.meshgrid(np.arange(-10, 10,1/10.0 ),np.arange(-10, 10,1/10.0))
    gx1l = gx1.flatten()
    gx2l = gx2.flatten()
    gx   = np.vstack((gx1l,gx2l)).T       
    gyhat = discriminant(gx,w0,w1,b)  
    gyhat = gyhat.reshape(gx1.shape)
    
    if(not all(gyhat.flatten()==0)):
        contours = plt.contour(gx1, gx2, gyhat, levels=[0], linewidths=2,colors=["k"]) 
        plt.imshow(gyhat, interpolation='nearest',extent=(-10, 10, -10, 10), aspect='auto', origin='lower', cmap=plt.cm.bwr_r, alpha=0.25)
        plt.clim(-10,10)
        plt.colorbar()
    
    plt.xlim(-10,10)
    plt.ylim(-10,10)
    plt.title("w0: %.2f w1:%2.f b: %.2f  Error Rate: %.2f"%(w0, w1, b, R))
    plt.xlabel("X0")
    plt.ylabel("X1")
    plt.grid(True)
    plt.legend(["Class 1","Class -1"],loc='upper left')
    plt.show()

ww0=widgets.FloatSlider(value=0.1,min=-1,max=1,step=.1,description="w0",continuous_update=False)
ww1=widgets.FloatSlider(value=0,min=-1,max=1,step=.1,description="w1",continuous_update=False)
wb=widgets.FloatSlider(value=0,min=-10,max=10,step=.1,description="b",continuous_update=False)
wd=widgets.Dropdown(options=[("Dataset 1"),("Dataset 2")],description="Dataset")
wn=widgets.IntSlider(value=0,min=-1,max=N,step=1,description="N",continuous_update=False)
wl=widgets.Checkbox(value=False,description="Learn")

out = interactive_output(plot_model, {"w0":ww0, "w1":ww1,"b":wb,"dataset":wd,"N":wn,"learn":wl});
out.layout.height = '400px'
box1=widgets.HBox([ww0,ww1,wb])
box2=widgets.HBox([wd,wn,wl])
ui=widgets.VBox([box1,box2])


## Linear Classifier Demo

In [147]:
display(out,ui)

Output(layout=Layout(height='400px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure…